### 1. Data source: Rashid SoH-from-EIS dataset

For this notebook I use the public dataset:

> Rashid, Muhammad; Faraji-Niri, Mona; Sansom, Jonathan; Sheikh, Muhammad; Widanage, Dhammika; Marco, James (2023),  
> **“Dataset for rapid state of health estimation of lithium batteries using EIS and machine learning: Training and validation”**,  
> *Data in Brief*, 48, 109157, doi: 10.1016/j.dib.2023.109157.  
> Original data: **“DIB_Data”**, Mendeley Data, V3, doi: 10.17632/mn9fb7xdx6.3 (license: **CC0 1.0**).

Dataset highlights:

- 25 cylindrical Li-ion cells, aged from **SoH 100% down to 80%** in 5 steps (100, 95, 90, 85, 80%).
- At each SoH stage, reference performance tests (capacity / SoH) and **electrochemical impedance spectroscopy (EIS)** at multiple SOC and temperature conditions.
- Designed specifically to study **fast SoH estimation from EIS with machine learning**.

In this notebook I will:

- Build a **tidy table of EIS measurements** with associated SoH, SOC and temperature.
- Engineer compact **impedance features** (real/imaginary parts, magnitude, phase at selected frequencies).
- Train and evaluate **SoH regression models** using these EIS features.

Folder structure in this repo (after placing the dataset under `./data/`):

- `../data/DIB_Data/`
  - `.csvfiles/`
    - `Capacity_Check/` — capacity check results as CSV/Excel files.
    - `EIS_Test/` — EIS measurements as Excel (`.xls`) files (used in this notebook).
  - `.matfiles/`
    - `Capacity_Check/` — capacity check results as MATLAB files.
    - `EIS_Test/` — EIS measurements as MATLAB files (not used here).
  - `WholeDataRealSOH.mat` — MATLAB table summarising all tests (not used here).
  - `CapacityVsCycleNumber.xlsx` — capacity vs cycle reference.
  - `Table 1.docx` — description of the experimental matrix and test conditions.


### 2. EIS files: indexing & loading

In this section I locate all EIS test files, extract their metadata from the filenames,
and load the spectra into a single long-format table.

#### 2.1 List EIS test files

EIS measurements are stored as Excel files under:

- `../data/DIB_Data/.csvfiles/EIS_Test/`

Each file corresponds to one EIS test at a specific (cell, SoH, SOC, temperature).

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

DATA_ROOT = Path("../data/DIB_Data")
EIS_XLS_DIR = DATA_ROOT / ".csvfiles" / "EIS_Test"

def parse_eis_filename(path: Path) -> dict:
    """
    Parse an EIS filename of the form:
    Cell02_95SOH_15degC_05SOC_9505.xls
    """
    stem = path.stem  # drop extension
    parts = stem.split("_")
    if len(parts) != 5:
        raise ValueError(f"Unexpected EIS filename format: {path.name}")

    cell_str, soh_str, temp_str, soc_str, cap_str = parts

    cell_id = int(cell_str.replace("Cell", ""))
    soh_pct = int(soh_str.replace("SOH", ""))
    temp_c = int(temp_str.replace("degC", ""))
    soc_pct = int(soc_str.replace("SOC", ""))
    capacity_code = int(cap_str)

    return {
        "filepath": path,
        "cell_id": cell_id,
        "soh_pct": soh_pct,
        "temp_c": temp_c,
        "soc_pct": soc_pct,
        "capacity_code": capacity_code,
    }

eis_files = sorted(EIS_XLS_DIR.glob("*.xls"))
len(eis_files)


360

#### 2.2 Parse metadata from filenames

Filenames follow the pattern:

`Cell02_95SOH_15degC_05SOC_9505.xls`

From which I extract:

- `cell_id`   → `02`
- `soh_pct`   → `95`
- `temp_c`    → `15`
- `soc_pct`   → `05`
- `capacity_code` → `9505` (numeric code related to capacity / SoH, kept as extra metadata)


In [2]:
eis_index_records = [parse_eis_filename(p) for p in eis_files]
eis_index_df = pd.DataFrame(eis_index_records)

eis_index_df.head()


,filepath,cell_id,soh_pct,temp_c,soc_pct,capacity_code
0,../data/DIB_Data/.csvfiles/EIS_Test/Cell02_95S...,2,95,15,5,9505
1,../data/DIB_Data/.csvfiles/EIS_Test/Cell02_95S...,2,95,15,20,9505
2,../data/DIB_Data/.csvfiles/EIS_Test/Cell02_95S...,2,95,15,50,9505
3,../data/DIB_Data/.csvfiles/EIS_Test/Cell02_95S...,2,95,15,70,9505
4,../data/DIB_Data/.csvfiles/EIS_Test/Cell02_95S...,2,95,15,95,9505


#### 2.3 Sanity checks on metadata

I check that:

- There are 360 EIS tests in total.
- SoH, SOC and temperature cover the expected grids.
- There are multiple cells, so I can later split by `cell_id`.

In [3]:
print("Number of EIS tests:", len(eis_index_df))

print("\nUnique cells:", sorted(eis_index_df["cell_id"].unique()))
print("\nUnique SoH levels:", sorted(eis_index_df["soh_pct"].unique()))
print("\nUnique temperatures (°C):", sorted(eis_index_df["temp_c"].unique()))
print("\nUnique SOC levels (%):", sorted(eis_index_df["soc_pct"].unique()))


Number of EIS tests: 360

Unique cells: [np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32)]

Unique SoH levels: [np.int64(80), np.int64(85), np.int64(90), np.int64(95), np.int64(100)]

Unique temperatures (°C): [np.int64(15), np.int64(25), np.int64(35)]

Unique SOC levels (%): [np.int64(5), np.int64(20), np.int64(50), np.int64(70), np.int64(95)]


#### 2.4 Load EIS spectra

Each EIS file contains one spectrum with three columns:

- column 1: frequency (Hz),
- column 2: real part of impedance Re(Z) (Ω),
- column 3: imaginary part of impedance Im(Z) (Ω).

I load all 360 files into a single long-format DataFrame where each row is a
single (cell, SoH, SOC, T, frequency) point:

- `cell_id, soh_pct, temp_c, soc_pct, capacity_code`
- `frequency_hz, z_real_ohm, z_imag_ohm`

In [4]:
example_path = eis_index_df.loc[0, "filepath"]
example_path


PosixPath('../data/DIB_Data/.csvfiles/EIS_Test/Cell02_95SOH_15degC_05SOC_9505.xls')

In [5]:
example_raw = pd.read_excel(example_path, header=None)
example_raw.head(), example_raw.shape


(         0        1        2
 0  10000.0  0.02995  0.03160
 1   7943.0  0.02827  0.02617
 2   6310.0  0.02704  0.02143
 3   5012.0  0.02599  0.01736
 4   3981.0  0.02499  0.01377,
 (61, 3))

In [7]:
def load_single_eis(path: Path) -> pd.DataFrame:
    """
    Load one EIS .xls file and return a DataFrame with columns:
    frequency_hz, z_real_ohm, z_imag_ohm.

    Any non-numeric rows (e.g. accidental headers) are dropped after
    coercing to numeric.
    """
    df = pd.read_excel(path, header=None)

    # Keep first three columns and assign names
    df = df.iloc[:, :3].copy()
    df.columns = ["frequency_hz", "z_real_ohm", "z_imag_ohm"]

    # Coerce to numeric and drop non-numeric rows
    for col in ["frequency_hz", "z_real_ohm", "z_imag_ohm"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df = df.dropna(subset=["frequency_hz", "z_real_ohm", "z_imag_ohm"])

    df = df.astype(
        {
            "frequency_hz": float,
            "z_real_ohm": float,
            "z_imag_ohm": float,
        }
    )

    return df


In [8]:
def load_all_eis(eis_index: pd.DataFrame) -> pd.DataFrame:
    """
    Load all EIS spectra listed in eis_index into a single long-format DataFrame.

    Output columns:
    - cell_id, soh_pct, temp_c, soc_pct, capacity_code
    - frequency_hz, z_real_ohm, z_imag_ohm
    """
    all_records = []

    for _, meta in eis_index.iterrows():
        spec = load_single_eis(meta["filepath"])

        # Attach metadata
        for col in ["cell_id", "soh_pct", "temp_c", "soc_pct", "capacity_code"]:
            spec[col] = meta[col]

        all_records.append(spec)

    return pd.concat(all_records, ignore_index=True)

eis_long_df = load_all_eis(eis_index_df)
eis_long_df.head()


,frequency_hz,z_real_ohm,z_imag_ohm,cell_id,soh_pct,temp_c,soc_pct,capacity_code
0,10000.0,0.02995,0.03160,2,95,15,5,9505
1,7943.0,0.02827,0.02617,2,95,15,5,9505
2,6310.0,0.02704,0.02143,2,95,15,5,9505
3,5012.0,0.02599,0.01736,2,95,15,5,9505
4,3981.0,0.02499,0.01377,2,95,15,5,9505


In [9]:
print(eis_long_df.shape)
eis_long_df.describe().T.head(10)


(21960, 8)


,count,mean,std,min,25%,50%,75%,max
frequency_hz,21960.0,797.097582,1951.144325,0.01000,0.316200,10.000000,316.200000,10000.00000
z_real_ohm,21960.0,0.029719,0.007403,0.02261,0.025330,0.027710,0.031280,0.08710
z_imag_ohm,21960.0,0.000229,0.007416,-0.03523,-0.002455,-0.001167,-0.000125,0.03542
cell_id,21960.0,18.291667,9.149339,2.00000,12.750000,19.500000,25.250000,32.00000
soh_pct,21960.0,90.000000,7.217043,80.00000,85.000000,90.000000,95.000000,100.00000
temp_c,21960.0,25.000000,8.165152,15.00000,15.000000,25.000000,35.000000,35.00000
soc_pct,21960.0,48.000000,32.650399,5.00000,20.000000,50.000000,70.000000,95.00000
capacity_code,21960.0,9034.750000,721.318964,8046.00000,8307.250000,9067.000000,9612.750000,10000.00000


In [10]:
n_freq_per_test = (
    eis_long_df.groupby(["cell_id", "soh_pct", "temp_c", "soc_pct"])
    ["frequency_hz"]
    .nunique()
)

n_freq_per_test.value_counts()


frequency_hz
61    360
Name: count, dtype: int64

#### 2.5 Final count of complete EIS tests

Here I summarise how many **distinct EIS tests** I have after loading and basic cleaning,
and how this compares to the dataset description:

- number of cells,
- number of EIS tests (unique combinations of cell, SoH, SOC, temperature),
- SoH / SOC / temperature grids,
- typical number of frequency points per spectrum.


In [12]:
# One row per unique EIS test (cell, SoH, T, SOC)
tests_df = (
    eis_long_df
    .groupby(["cell_id", "soh_pct", "temp_c", "soc_pct"], as_index=False)
    .agg(
        n_points=("frequency_hz", "size"),
        n_freq=("frequency_hz", "nunique"),
    )
)

n_cells = tests_df["cell_id"].nunique()
n_tests = len(tests_df)

soh_levels = sorted(tests_df["soh_pct"].unique())
soc_levels = sorted(tests_df["soc_pct"].unique())
temp_levels = sorted(tests_df["temp_c"].unique())

# If all spectra have the same number of frequency points, report that
if tests_df["n_freq"].nunique() == 1:
    freq_points_per_spectrum = int(tests_df["n_freq"].iloc[0])
else:
    freq_points_per_spectrum = f"varies ({tests_df['n_freq'].min()}–{tests_df['n_freq'].max()})"

summary_table = pd.DataFrame(
    {
        "n_cells": [n_cells],
        "n_eis_tests": [n_tests],
        "soh_levels": [", ".join(str(x) for x in soh_levels)],
        "soc_levels": [", ".join(str(x) for x in soc_levels)],
        "temp_levels": [", ".join(str(x) for x in temp_levels)],
        "freq_points_per_spectrum": [freq_points_per_spectrum],
    }
)

summary_table


,n_cells,n_eis_tests,soh_levels,soc_levels,temp_levels,freq_points_per_spectrum
0,24,360,"80, 85, 90, 95, 100","5, 20, 50, 70, 95","15, 25, 35",61
